# Pump Example Notebook
This script shows basic I/O operations that can be performed with this toolkit, as well as some of the basic model and simulation visualization and analysis features.

This script runs these basic operations on the simple model defined in ex_pump.py.

```
Copyright © 2024, United States Government, as represented by the Administrator of the National Aeronautics and Space Administration. All rights reserved.

The “"Fault Model Design tools - fmdtools version 2"” software is licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.
```


In [1]:
from ex_pump import Pump

import fmdtools.sim.propagate as propagate
from fmdtools import analyze as an

from IPython.display import HTML

ModuleNotFoundError: No module named 'ex_pump'

### Initial Model Checks
Before seeing how faults propagate, it's useful to see that the model structure is set up correctly and that the system performs as expected.

The 'track' argument specifies what model attributes to log. Specifying `all` will log all attributes.

In [2]:
mdl = Pump(track='all')

One of the easiest ways to visualize this is by viewing the model in the repl, which shows thes states and modes of the component functions and flows.

In [3]:
mdl

#### Model Structure Visualization

To check that the simulation structures are set up right, the FunctionArchitectureGraph class lets us visualize the function/flow relationships in the model. This helps us answer the questions:
   - are all functions on the graph?
   - are the functions connected with the correct flows?

In [4]:
from fmdtools.define.architecture.function import FunctionArchitectureGraph

In [5]:
a = FunctionArchitectureGraph(mdl)
fig, ax = a.draw(figsize=(8,6))

Note that a variety of different Classes can be used for model structure visualization, inlcluding `ModelFxnGraph`, `ModelFlowGraph`, and `ModelTypeGraph`. This is further explained in `examples/rover/Model_Structure_Visualization_Tutorial.ipynb`

In [6]:
from fmdtools.define.architecture.function import FunctionArchitectureFxnGraph
b=FunctionArchitectureFxnGraph(mdl)
fig = b.draw(figsize=(8,6))

#### Nominal Run

The next code runs the model in the nominal state to check to see that the model has been defined correctly.
This helps us verify:
   - if any faults occur in the nominal scenario
   - if the progression of states proceeds as desired over time.

The following code runs the model with no faults to let us do that. The inputs are:
- mdl (the model we imported at the start of the script)
- desired_result (str/list/dict describing what to return in result)
- **kwargs (see docs)

The outputs are:
- result (a `Result` object defined in `analyze.result`)
- mdlhist (a `History` object defined in `analyze.result`)

Both `Result` and `History` have a number of methods that can be readily used to process and analyze simulation results. See:
- [Result documentation](https://nasa.github.io/fmdtools/docs-source/fmdtools.analyze.html#fmdtools.analyze.result.Result)
- [History documentation](https://nasa.github.io/fmdtools/docs-source/fmdtools.analyze.html#fmdtools.analyze.result.History)

Many different properties can be requested given the `desired_result` argument ([see full list here](https://nasa.github.io/fmdtools/docs-source/fmdtools.sim.html#fmdtools.sim.propagate.sim_kwargs)). In this case, we pass a dict with key `graph` and a value `FunctionArchitectureGraph` specifying that we want it to give us a graph view of the `Model`.

In [7]:
result, mdlhist=propagate.nominal(mdl, desired_result={'graph': FunctionArchitectureGraph})

Here we can see where it is in the Result:

In [8]:
result.keys()

In [9]:
result.graph

With these results, we can now plot the graph of results resgraph using:

In [10]:
fig = result.graph.draw(figsize=(6,4))

As can be seen, this gives a graphical representation of the functional model with the various flows. Since all of the functions are *grey*, no faults were accidentally introduced in this run.


A model history is additionally returned given our specified tracking options. If none are provided, the `default_track` variable in the `Model` is used (which in this case is set to `all`). See below:

In [11]:
mdlhist

We can further look at the states of the model over time using `History.plot_line`:

In [12]:
fig, ax = mdlhist.plot_line('flows.wat_1.s.flowrate', 'flows.wat_2.s.flowrate', 'i.on', 'flows.sig_1.s.power')

As we can see, the state of these flows does exactly what we would expect--when the switch turns on at $t=5$, the pump switches on and there is a flow of water in and out of the model.

### History
If we want to see this data in tabular form, we can use `fp.tabulate.hist()`:

In [13]:
nominal_histtable = mdlhist.as_table()
nominal_histtable[:10] #only displaying 10 

This table is a pandas dataframe. We can save this dataframe to a .csv using `nominal_histtable.to_csv("filename.csv")`

### Propagating and Viewing Results for Individual Faults
It is often necessary to see how the system reacts to individual faults. This can gives us better understanding of how the system behaves under individual faults and can let us iterate with the model better.

The following code runs the model with a single fault in a single function. In this case, we are initiating a short in the 'Move Water' function at 10 hours into the system's operation.

The inputs are:
- `mdl` (the model we imported at the start of the script)
- `function` (the function the fault we're interested in propagating occurs in)
- `faultmode` (the fault to initiate)
- `time` (the time when the fault is initiated)
- **kwargs )

The outputs are (the same as propogate.nominal):
- `results` (a dictionary corresponding to `desired_result`)
- `mdlhist` (the states of the model over time)

In [14]:
endresults, mdlhist=propagate.one_fault(mdl, 'move_water', 'short', time=10, 
                                        desired_result=['graph','endclass','endfaults'])

Now mdlhist has double the number of entries--those corresponding to the nominal and faulty scenarios.

In [15]:
mdlhist

`History.get_degraded_hist` compares the results over time so we can see what functions and flows were degraded over time. We can then use the summary to view a list of the functions and flows that were impacted over time.

In [16]:
deghist = mdlhist.get_degraded_hist(*mdl.fxns, *mdl.flows)

In [17]:
deghist

In [18]:
deghist.as_table()

`endresults` however, keeps the endresult for the faulty scenario alone, as shown:

In [19]:
endresults

However, the graph view now has information about degradations between the faulty and nominal runs, along with fault information:

In [20]:
fig = endresults.graph.draw()

In [21]:
endresults.graph.g.nodes['pump.fxns.move_water']

As can be seen, at the final t, the short causes a degraded flow of electricity as well as a fault in the Import EE function. 

**However**, we would imagine that the short would cause the water to stop moving also--so why is it not red?

The answer is that by default the degradations shown in the graph are shown at the **final time**, which is the same both for the failed model and the nominal model, since the pump is switched "off." In this case we might be more interested in looking at how the graph looks in operation, rather than at the end.

We can do that that in two ways:
- by specifying a different time to fetch the graph from (e.g., `desired_result={10:'graph'}`, or
- by reconstructing the based on the history of the plot, as shown below:

In [22]:
mg = FunctionArchitectureGraph(mdl)

To do this, we first need to track more states than have been specified to track in the model. The easiest way to do this is to set `track='all'`.

In [23]:
endresults, mdlhist_full=propagate.one_fault(mdl, 'move_water', 'short', time=10, track='all',
                                        desired_result=['graph','endclass','endfaults'])

We can then plot the state at any time in the history using `mg.draw_from`.

In [24]:
fig, ax = mg.draw_from(15, mdlhist_full)

As shown, this version has the degradation of the water, since at this time the off-nominal state is different from the nominal (no flow).

We can view an animation over time using:

In [25]:
from IPython.display import HTML
ani = mg.animate(mdlhist_full)
HTML(ani.to_jshtml())

Note that if only a partial history is given, only partial results will be displayed (see below).

In [26]:
fig, ax = mg.draw_from(15, mdlhist, figsize=(6,4), withlegend=False)

We can also plot the states of this against the nominal run using:

In [27]:
mdlhist

In [28]:
fig, axs = mdlhist.plot_line('flows.ee_1.s.current', 'flows.wat_2.s.flowrate', 
                             title="Response of Pump to Short", time_slice=10, legend_loc=False, title_padding=0.1)

As you can see, the system begins nominal until the fault is injected at $t=10$. At this moment, not only are the electrical energy flows degraded, the flow of water is degraded also. However, at $t=55$ when the system is supposed to be turned off, this flow of water is no longer "degraded" because it is in the same state as the nominal system.

We can look at a table of to see more precisely what happened (and export, if needed). Note that we need to give the plotting function the mode ('short') and the time for it to plot properly.

In [29]:
mdlhist.as_table()

Here we can see that the short dropped the voltage to zero, (this was because an open circuit resulted in the Import EE function), causing the water to stop flowing. Below, we use the processed model history to show the faults and *degradation* of states over time. In this case, 1 means nominal while 0 means degraded.

In [30]:
deghist = mdlhist_full.get_degraded_hist(*mdl.fxns, *mdl.flows)

In [31]:
deghist.as_table()[:20]

We can also look at the faults over time...

In [32]:
faulthist = mdlhist_full.get_faulty_hist(*mdl.fxns)
faulthist.as_table()[0:20]

We can also look at statistics of degradation over time using:

In [33]:
summ = mdlhist_full.get_fault_degradation_summary(*mdl.fxns, *mdl.flows)
summ.faulty

In [34]:
summ.degraded

#### Blockage Fault

We can also look at other faults. The results below are for a blockage of the pipe. In this case we're only interested in the effect on the water going through, so only those flows are tracked.

In [35]:
endresults2, mdlhist2=propagate.one_fault(mdl, 'export_water', 'block', 
                                          time=10, desired_result=['endclass', 'graph', 'endfaults'])
summ = mdlhist_full.get_fault_degradation_summary(*mdl.fxns, *mdl.flows)

In [36]:
summ.faulty

In [37]:
summ.degraded

In [38]:
fig, ax = endresults2.graph.draw(figsize=(6,4), withlegend=False)

In [39]:
mdlhist

In [40]:
fig, axs = mdlhist2.plot_line('flows.ee_1.s.current', 'flows.wat_2.s.flowrate',
                              title = 'Response of Pump to blockage', time_slice=10, legend_loc=False, title_padding=0.1)

### Visualization of resilience metrics
We can also use the processed time history to now make visualizations of the resilience of the system over time. 

Here we calculate the percent time the simulation was decraded over the simulation interval:

In [41]:
fxns_and_flows = [*mdl.get_roles_as_dict("fxns", "flows", flex_prefixes=True)]
deghist = mdlhist_full.get_degraded_hist(*fxns_and_flows)
exp = deghist.get_metrics()

In [42]:
exp

These metrics (and others like them) can then be overlayed as a heatmap using `set_heatmap`.

In [43]:
mg = FunctionArchitectureGraph(mdl)
mg.set_heatmap({mdl.name+"."+k: v for k, v in exp.items()})
mg.draw()

Network metrics can also be overlaid on the graphs:

In [44]:
mg = FunctionArchitectureGraph(mdl)
fig, ax = mg.plot_high_degree_nodes()

In [45]:
mg = FunctionArchitectureGraph(mdl)
fig, ax = mg.plot_bridging_nodes()

### Running a List of Faults
Finally, to get the results of all of the single-fault scenarios defined in the model, we can run them all at once using the `single_faults()` function. Note that this will propagate faults based on the times vector put in the model it will propogate the faults at the begining, end, and at t=15 and t=15. This function only takes in the model mdl and outputs two similar kinds of output--resultsdict (the results in a python dictionary) and resultstab (the results in a nice tabular form). 

Note that the rates provide for this table do not use the opportunity vector information, instead using the assumption that the fault scenario has the rate provided over the entire simulation.

See below:

In [46]:
endclasses, mdlhists=propagate.single_faults(mdl, staged=True, track="all")

We can visualize the metrics for each scenario using `Result.create_simple_fmea`

In [47]:
endclasses.create_simple_fmea()

We can see corresponding degradations using `tabulate.result_summary.fmea()`

In [48]:
fullfmea = an.tabulate.result_summary_fmea(endclasses, mdlhists, *mdl.fxns, *mdl.flows)
fullfmea[:10]

### Running a Fault Sampling Approach
Note that only gives accurate results for costs and fault responses--in order to get an accurate idea of *expected cost*, we instead run a FaultSample or SampleApproach, which develops an underlying probability model for faults. See below.

In [49]:
from fmdtools.sim.sample import FaultDomain, FaultSample

fd = FaultDomain(mdl)
fd.add_all()
fd

In [50]:
from fmdtools.analyze.phases import PhaseMap
fs = FaultSample(fd, phasemap=PhaseMap(mdl.sp.phases))
fs.add_fault_phases()
fs

In [51]:
endclasses, mdlhists=propagate.fault_sample(mdl, fs, staged=True, track="all")
simplefmea = endclasses.create_simple_fmea() #note the costs are the same, but the rates and expected costs are not
simplefmea[:5]

We can now summarize the risks of faults over the operational phases and overall using the `FMEA` class:

In [52]:
phasefmea = an.tabulate.FMEA(endclasses, fs, group_by=('function', 'fault', 'phase'))
phasefmea.as_table()

In [53]:
summfmea = an.tabulate.FMEA(endclasses, fs)
summfmea.as_table()

#### History visualization

We can further overlay expected resilience metrics on the model over fault scenarios using various methods provided with `History` and `Result`.

Below we get the expected values of the history values:

In [54]:
hist_expected = mdlhists.get_expected(app=fs, with_nominal=True)

Next we get the expected degradations:

In [55]:
deg=hist_expected.get_degraded_hist(*fxns_and_flows, nomhist=mdlhists.nominal)

In [56]:
deg

In [57]:
import numpy as np
heatmap = deg.get_metrics(metric=np.mean)


These metrics can in turn be overlayed on the graph using `set_heatmap`.

In [58]:
from fmdtools.define.architecture.function import FunctionArchitectureGraph
mg = FunctionArchitectureGraph(mdl)
mg.set_heatmap({mdl.name+"."+k: v for k, v in exp.items()})
mg.draw()

## Save/Load

In detailed simulations, running a lot of computational simulations can take a considerable amount of time. As a result, it becomes impractical to run a new simulation every time one wishes to analyse its data. Results from fmdtools simulations (endclasses or mdlhists) can be saved as pickle, csv, or json files in this instance using either:
- `Result.save` or `History.save` or 
- passing a save_args dictionary to the respective propagate functions (e.g., {'endclass':{'filename':'file.pkl','overwrite':True})

and then loaded using:
- `Result.load` or `History.load`

In [59]:
mdlhists

This saves a history to a file:

In [60]:
mdlhists.save("example_mdlhist.npz", overwrite=True)

And this loads this history:

In [61]:
mdlhists_saved  = an.history.History.load("example_mdlhist.npz")

In [62]:
mdlhists_saved 

Note that there are different trade-offs to using different file formats:
- `npz` is the serialization format provided in numpy, which is fast but not human readable
- `csv` outputs as comma separated values, which are slower and less robust (in terms of data types), but are human-readable and can be opened in a spreadsheet software like excel.
- `json` is similar to csv, but is less human readable.